In [1]:
print("all ok")

all ok


In [2]:
pip install langchain


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pinecone


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -U langchain-community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.4 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.7 MB/s eta 0:00:01
   ------------------------ --------------- 1.6/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 2.0 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.5 MB 2.1 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.5 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------------------------------- -------- 0.8/1.0 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 4.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
 

In [5]:
pip install langchain pypdf


Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA


import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\Asus\Desktop\Call_Agent_AI\Call_Agent_AI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [8]:
extracted_data = load_pdf("../data")

In [9]:
extracted_data

[Document(metadata={'producer': 'Skia/PDF m136 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Sales Agent - Customer Conversation (TV Selling)', 'source': '..\\data\\Sales Agent.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Sales  Agent  -  Customer  Conversation  (TV  Selling)  \nAgent:  \n \nGood\n \nafternoon!\n \nAm\n \nI\n \nspeaking\n \nwith\n \nMr.\n \nRakesh\n \nSharma?\n \nCustomer:  \n \nYes,\n \nspeaking.\n \nAgent:  \n \nHi\n \nMr.\n \nRakesh,\n \nthis\n \nis\n \nPriya\n \nfrom\n \nBrightView\n \nElectronics\n.\n \nI’m\n \ncalling\n \nto\n \nshare\n \nan\n \nexciting\n \noffer\n \non\n \nour\n \nlatest\n \n55-inch\n \nSmart\n \n4K\n \nLED\n \nTV.\n \nMay\n \nI\n \ntake\n \na\n \nminute\n \nof\n \nyour\n \ntime?\n \nCustomer:  \n \nOkay,\n \nsure.\n \nAgent:  \n \nThank\n \nyou!\n \nThe\n \nTV\n \ncomes\n \nwith\n \nUltra\n \nHD\n \nresolution,\n \nbuilt-in\n \nWi-Fi,\n \npre-installed\n \napps\n \nlike\n \nNetflix\n \nand\n \

In [10]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 3


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [13]:

embeddings = download_hugging_face_embeddings()

C:\Users\Asus\AppData\Local\Temp\ipykernel_10532\297195633.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [14]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [15]:
query_result=embeddings.embed_query("Hello world")
print("length",len(query_result))

length 384


In [20]:
# query_result

In [20]:
import os

from dotenv import load_dotenv

In [21]:

load_dotenv()

True

In [22]:
import pinecone

In [23]:
pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install --upgrade pinecone pinecone-plugin-assistant

^C
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.2.3 requires pinecone<6.0.0,>=5.4.0, but you have pinecone 6.0.2 which is incompatible.


  Using cached pinecone-6.0.2-py3-none-any.whl.metadata (9.0 kB)
Using cached pinecone-6.0.2-py3-none-any.whl (421 kB)
  Attempting uninstall: pinecone
    Found existing installation: pinecone 5.4.2
    Uninstalling pinecone-5.4.2:
      Successfully uninstalled pinecone-5.4.2


In [25]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

In [18]:
from pinecone import Pinecone
from pinecone import ServerlessSpec

In [26]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [27]:

pc = Pinecone(api_key=PINECONE_API_KEY)

In [30]:
index_name = "call-agent-ai"

In [31]:

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': '1b17447e40aacd0cb8dc5ad2b03c2f20', 'date': 'Fri, 21 Mar 2025 17:18:07 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [32]:
pip install langchain_pinecone

  Using cached pinecone-5.4.2-py3-none-any.whl.metadata (19 kB)
Using cached pinecone-5.4.2-py3-none-any.whl (427 kB)
  Attempting uninstall: pinecone
    Found existing installation: pinecone 6.0.2
    Uninstalling pinecone-6.0.2:
      Successfully uninstalled pinecone-6.0.2
Note: you may need to restart the kernel to use updated packages.


In [33]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [34]:
docsearch

In [25]:
docsearch

In [35]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [36]:
retrieved_docs = retriever.invoke("what is call agent?")

In [37]:
retrieved_docs 

[Document(id='283e9e90-09e3-4ef4-a58f-3ce744ad2904', metadata={'creationdate': '', 'creator': 'PyPDF', 'page': 0.0, 'page_label': '1', 'producer': 'Skia/PDF m136 Google Docs Renderer', 'source': '..\\data\\Sales Agent.pdf', 'title': 'Sales Agent - Customer Conversation (TV Selling)', 'total_pages': 1.0}, page_content='Sales  Agent  -  Customer  Conversation  (TV  Selling)  \nAgent:  \n \nGood\n \nafternoon!\n \nAm\n \nI\n \nspeaking\n \nwith\n \nMr.\n \nRakesh\n \nSharma?\n \nCustomer:  \n \nYes,\n \nspeaking.\n \nAgent:  \n \nHi\n \nMr.\n \nRakesh,\n \nthis\n \nis\n \nPriya\n \nfrom\n \nBrightView\n \nElectronics\n.\n \nI’m\n \ncalling\n \nto\n \nshare\n \nan\n \nexciting\n \noffer\n \non\n \nour\n \nlatest\n \n55-inch\n \nSmart\n \n4K\n \nLED\n \nTV.\n \nMay\n \nI\n \ntake\n \na\n \nminute\n \nof\n \nyour\n \ntime?\n \nCustomer:  \n \nOkay,\n \nsure.\n \nAgent:  \n \nThank\n \nyou!\n \nThe\n \nTV\n \ncomes\n \nwith'),
 Document(id='6c753d3b-e86b-4d3d-8f2d-84a491da15b4', metadata={'cr

In [38]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [39]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [40]:
import google.generativeai as genai

In [43]:
genai.configure(api_key=os.getenv('GEMINI_API_KEY')) 

In [45]:
def query_gemini_with_pinecone(user_query):
    # Perform similarity search in Pinecone
    search_results = docsearch.similarity_search(user_query, k=2)

    # Extract context from search results
    context_texts = [doc.page_content for doc in search_results]
    context_combined = '\n\n'.join(context_texts)

    # Generate answer using Gemini
    model = genai.GenerativeModel('gemini-1.5')
    prompt = PROMPT.format(context=context_combined, question=user_query)
    response = model.generate_content(prompt)

    return response.text

# Test the retrieval and generation flow
user_query = "what is debate?"
response = query_gemini_with_pinecone(user_query)
print("User Query:", user_query)
print("Response:", response)

NotFound: 404 models/gemini-1.5 is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

In [ ]:
search_results = docsearch.similarity_search(user_query, k=2)

    # Extract context from search results
context_texts = [doc.page_content for doc in search_results]
context_combined = '\n\n'.join(context_texts)

In [44]:
model = genai.GenerativeModel('gemini-pro')
prompt = PROMPT.format(context=context_combined, question=user_query)
response = model.generate_content(prompt)

NameError: name 'context_combined' is not defined

In [ ]:
from langchain_community.llms import CTransformers

In [ ]:
# !pwd
# llm=CTransformers(model="../models/llama_model_1.bin",  
#                   model_type="llama",
#                   config={'max_new_tokens':512,
#                           'temperature':0.8})

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
llm

CTransformers(client=<ctransformers.llm.LLM object at 0x000002A612122510>, model='../models/laama.bin', model_type='llama', config={'max_new_tokens': 512, 'temperature': 0.8})

In [ ]:
from langchain.schema import Document

In [46]:
from langchain.chains import RetrievalQA

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

C:\Users\Asus\AppData\Local\Temp\ipykernel_26744\4216317267.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})


Response :  Debating is an art form, just like playing chess, you need to learn the moves, mastering different techniques and strategies, and practice makes perfect!


ProtocolError: Failed to connect; did you specify the correct index name?